In [69]:
import pandas
import numpy
from scipy import stats

## Hybrid sample

In [70]:
filepath = 'D:/BG/Data/Processing/4_data_hybrid.txt'
hybrid = pandas.read_csv(filepath, sep='\t')
print(hybrid.shape[0],'job postings in hybrid sample')
hybrid.columns

212821 job postings in hybrid sample


Index(['BGTJobId', 'JobDate', 'CleanTitle', 'CanonTitle', 'SOC', 'SOCName',
       'ONET', 'ONETName', 'Employer', 'Sector', 'SectorName', 'NAICS3',
       'NAICS4', 'NAICS5', 'NAICS6', 'City', 'County', 'State', 'Lat', 'Lon',
       'Edu', 'MaxEdu', 'Degree', 'MaxDegree', 'Exp', 'MaxExp', 'MinSalary',
       'MaxSalary', 'MinHrlySalary', 'MaxHrlySalary', 'PayFrequency',
       'SalaryType', 'JobHours', 'SOC2', 'SOC4', 'SOC5', 'SK',
       'Employer_clean', 'n_terms', 'SK_without', 'TECH', 'occupation',
       'firm1', 'firm2', 'firm', 'plant', 'plant_all', 'plant_5',
       'development', 'materials', 'design', 'inventory', 'tooling',
       'automation', 'production', 'maintenance', 'technical',
       'administrative', 'management', 'finance', 'business', 'data',
       'software', 'office', 'ml', 'cognitive', 'creativity', 'social',
       'character', 'customer', 'writing', 'nonroutine analytic',
       'nonroutine manual', 'routine cognitive', 'routine manual',
       'sequential

In [71]:
mydf = hybrid.copy()
mydf['Count of Terms']=[len(job.split(',')) for job in mydf.SK_without.tolist()]
mydf['Engineering'] = mydf['development']+mydf['materials']+mydf['design']
mydf['Operations'] = mydf['inventory']+mydf['tooling']+mydf['automation']+mydf['production']+mydf['maintenance']
mydf['Support'] = mydf['administrative']+mydf['management']+mydf['finance']+mydf['business']+mydf['data']+mydf['software']+mydf['office']+mydf['ml']+mydf['technical']
mydf['Reasoning'] = mydf['cognitive']+mydf['creativity']
mydf['People'] = mydf['social']+mydf['character']+mydf['customer']+mydf['writing']
annotation = ['Engineering','development','design','materials','Operations','inventory','tooling','automation','production','maintenance','Support','administrative','management','finance','business','data','software','office','ml','technical','Reasoning','cognitive','creativity','People','social','character','customer','writing']
for a in annotation:
  mydf[a] = (mydf[a]*mydf['Count of Terms'].sum())/(mydf['Count of Terms']*mydf[a].sum())

## Statistics for all occupations

In [72]:
df = mydf[['TECH'] + annotation]

t = []
for a in annotation:
  t.append([a,stats.ttest_ind(df.loc[df.TECH=='AM',a],df.loc[df.TECH=='TM',a],equal_var=False)[1]])
tdf = pandas.DataFrame(t,columns=['annotation','p-val'])
tdf.set_index('annotation',inplace=True)

mymean = df.groupby(by=['TECH']).mean().T
mymean.index.name = 'annotation'
mymean['AM-TM'] = mymean['AM'] - mymean['TM']

mymean['N-AM'] = N = len(mydf.loc[mydf.TECH=='AM'])
mymean['N-TM'] = N = len(mydf.loc[mydf.TECH=='TM'])

table1 = pandas.merge(left=mymean,right=tdf,how='inner',left_on=['annotation'],right_on=['annotation'])
table1['statistics'] = 'Mean'
table1.set_index('statistics',append=True,inplace=True)

mysd = df.groupby(by=['TECH']).std(ddof=0).T
mysd.index.name = 'annotation'
mysd.reset_index(inplace=True)
mysd['statistics'] = 'SD'
mysd.set_index(['annotation','statistics'],inplace=True)
mysd['p-val'] = numpy.nan
table1 = pandas.concat([table1,mysd],axis=0).sort_index(kind='merge').reindex(annotation,axis=0,level='annotation')
table1.columns = pandas.MultiIndex.from_product([['All'],table1.columns])

postings = mydf.groupby(['TECH']).nunique()[['BGTJobId']].T
postings.rename_axis([None],axis=1,inplace=True)
postings.columns = pandas.MultiIndex.from_product([['All'],postings.columns])
postings.index = pandas.MultiIndex.from_product([['Number of job postings'],['']])
plants = mydf.groupby(['TECH']).nunique()[['plant']].T
plants.rename_axis([None],axis=1,inplace=True)
plants.columns = pandas.MultiIndex.from_product([['All'],plants.columns])
plants.index = pandas.MultiIndex.from_product([['Number of plants'],['']])
table1 = pandas.concat([table1,postings,plants],axis=0)
table1

All                              \
                                            AM     AM-TM    N-AM      N-TM   
annotation             statistics                                            
Engineering            Mean           1.796460  0.785225  4638.0  208183.0   
                       SD             1.506160       NaN     NaN       NaN   
development            Mean           1.653261  0.647780  4638.0  208183.0   
                       SD             2.125584       NaN     NaN       NaN   
design                 Mean           1.651273  0.669212  4638.0  208183.0   
                       SD             2.083312       NaN     NaN       NaN   
materials              Mean           2.345054  1.264174  4638.0  208183.0   
                       SD             4.219040       NaN     NaN       NaN   
Operations             Mean           1.138939  0.087528  4638.0  208183.0   
                       SD             1.122574       NaN     NaN       NaN   
inventory              Mean           0.568781 -0.415132  4638.0  208183.0   
                       SD             1.455696       NaN     NaN       NaN   
tooling                Mean           1.445359  0.130330  4638.0  208183.0   
                       SD            26.030696       NaN     NaN       NaN   
automation             Mean           2.898986  1.875994  4638.0  208183.0   
                       SD             6.862909       NaN     NaN       NaN   
production             Mean           1.559953  0.529747  4638.0  208183.0   
                       SD             1.927242       NaN     NaN       NaN   
maintenance            Mean           0.753594 -0.414489  4638.0  208183.0   
                       SD             1.679864       NaN     NaN       NaN   
Support                Mean           0.771135 -0.179255  4638.0  208183.0   
                       SD             0.668609       NaN     NaN       NaN   
administrative         Mean           1.371988  0.268008  4638.0  208183.0   
                       SD             3.712744       NaN     NaN       NaN   
management             Mean           0.863459 -0.157303  4638.0  208183.0   
                       SD             1.955991       NaN     NaN       NaN   
finance                Mean           0.753330 -0.217780  4638.0  208183.0   
                       SD             1.699604       NaN     NaN       NaN   
business               Mean           0.504726 -0.484756  4638.0  208183.0   
                       SD             1.588447       NaN     NaN       NaN   
data                   Mean           0.594837 -0.384824  4638.0  208183.0   
                       SD             2.916470       NaN     NaN       NaN   
software               Mean           0.708443 -0.159334  4638.0  208183.0   
                       SD             1.224541       NaN     NaN       NaN   
office                 Mean           0.762418 -0.170171  4638.0  208183.0   
                       SD             1.526695       NaN     NaN       NaN   
ml                     Mean           1.196098  0.413450  4638.0  208183.0   
                       SD             5.493493       NaN     NaN       NaN   
technical              Mean           0.615838 -0.381366  4638.0  208183.0   
                       SD             2.302044       NaN     NaN       NaN   
Reasoning              Mean           1.390865  0.410320  4638.0  208183.0   
                       SD             1.489409       NaN     NaN       NaN   
cognitive              Mean           1.371762  0.386135  4638.0  208183.0   
                       SD             1.585454       NaN     NaN       NaN   
creativity             Mean           1.492963  0.539579  4638.0  208183.0   
                       SD             4.150613       NaN     NaN       NaN   
People                 Mean           0.852389 -0.137489  4638.0  208183.0   
                       SD             0.688802       NaN     NaN       NaN   
social                 Mean           0.929739 -0.077609  4638.0 

## Statistics by occupation

In [73]:
occupations = ['Manager','Engineer','Technician','Operator']
df = mydf[['TECH','occupation'] + annotation]

t = []
for a in annotation:
  for o in occupations:
    t.append([a,o,stats.ttest_ind(df.loc[(df.TECH=='AM') & (df.occupation==o),a],df.loc[(df.TECH=='TM') & (df.occupation==o),a],equal_var=False)[1]])
tdf = pandas.DataFrame(t,columns=['annotation','occupations','p-val'])
tdf.set_index(['annotation','occupations'],inplace=True)

mymean = df.groupby(by=['occupation','TECH']).mean().unstack(level=0).T
mymean.index.names = ['annotation','occupations']
mymean['AM-TM'] = mymean['AM'] - mymean['TM']

mymean.loc[pandas.IndexSlice[:,'Manager'],'N-AM'] = len(mydf.loc[(mydf.TECH=='AM')&(mydf.occupation=='Manager')])
mymean.loc[pandas.IndexSlice[:,'Engineer'],'N-AM'] = len(mydf.loc[(mydf.TECH=='AM')&(mydf.occupation=='Engineer')])
mymean.loc[pandas.IndexSlice[:,'Technician'],'N-AM'] = len(mydf.loc[(mydf.TECH=='AM')&(mydf.occupation=='Technician')])
mymean.loc[pandas.IndexSlice[:,'Operator'],'N-AM'] = len(mydf.loc[(mydf.TECH=='AM')&(mydf.occupation=='Operator')])
mymean.loc[pandas.IndexSlice[:,'Manager'],'N-TM'] = len(mydf.loc[(mydf.TECH=='TM')&(mydf.occupation=='Manager')])
mymean.loc[pandas.IndexSlice[:,'Engineer'],'N-TM'] = len(mydf.loc[(mydf.TECH=='TM')&(mydf.occupation=='Engineer')])
mymean.loc[pandas.IndexSlice[:,'Technician'],'N-TM'] = len(mydf.loc[(mydf.TECH=='TM')&(mydf.occupation=='Technician')])
mymean.loc[pandas.IndexSlice[:,'Operator'],'N-TM'] = len(mydf.loc[(mydf.TECH=='TM')&(mydf.occupation=='Operator')])

table2 = pandas.merge(left=mymean,right=tdf,how='inner',left_on=['annotation','occupations'],right_on=['annotation','occupations'])
table2['statistics'] = 'Mean'
table2.set_index('statistics',append=True,inplace=True)

mysd = df.groupby(by=['occupation','TECH']).std(ddof=0).unstack(level=0).T
mysd.index.names = ['annotation','occupations']
mysd['statistics'] = 'SD'
mysd.set_index('statistics',append=True,inplace=True)
mysd['p-val'] = numpy.nan

table2 = pandas.concat([table2,mysd],axis=0).sort_index(kind='merge').reindex(annotation,axis=0,level='annotation').unstack('occupations').swaplevel(0,1,axis=1).reindex(occupations,axis=1,level=0)
table2.rename_axis([None,None],axis=1,inplace=True)

postings = mydf.groupby(['occupation','TECH']).nunique()[['BGTJobId']].reindex(occupations, level='occupation').T
postings.rename_axis([None,None],axis=1,inplace=True)
postings.index = pandas.MultiIndex.from_product([['Number of job postings'],['']])
plants = mydf.groupby(['occupation','TECH']).nunique()[['plant']].reindex(occupations, level='occupation').T
plants.rename_axis([None,None],axis=1,inplace=True)
plants.index = pandas.MultiIndex.from_product([['Number of plants'],['']])
table2 = pandas.concat([table2,postings,plants],axis=0).reindex(occupations,axis=1,level=0)
table2

Manager                            \
                                           AM     AM-TM   N-AM     N-TM   
annotation             statistics                                         
Engineering            Mean          0.857864  0.425481  981.0  88819.0   
                       SD            1.118225       NaN    NaN      NaN   
development            Mean          0.956899  0.373930  981.0  88819.0   
                       SD            1.785380       NaN    NaN      NaN   
design                 Mean          0.698259  0.360672  981.0  88819.0   
                       SD            1.312923       NaN    NaN      NaN   
materials              Mean          1.008726  0.648549  981.0  88819.0   
                       SD            2.720594       NaN    NaN      NaN   
Operations             Mean          0.634610  0.003514  981.0  88819.0   
                       SD            0.929923       NaN    NaN      NaN   
inventory              Mean          0.631094 -0.694956  981.0  88819.0   
                       SD            1.885894       NaN    NaN      NaN   
tooling                Mean          0.000000 -0.533118  981.0  88819.0   
                       SD            0.000000       NaN    NaN      NaN   
automation             Mean          1.242551  0.996256  981.0  88819.0   
                       SD            4.024428       NaN    NaN      NaN   
production             Mean          0.828816  0.393475  981.0  88819.0   
                       SD            1.565202       NaN    NaN      NaN   
maintenance            Mean          0.203702 -0.001294  981.0  88819.0   
                       SD            0.694632       NaN    NaN      NaN   
Support                Mean          1.083069 -0.063414  981.0  88819.0   
                       SD            0.811497       NaN    NaN      NaN   
administrative         Mean          2.976456  1.742861  981.0  88819.0   
                       SD            5.764335       NaN    NaN      NaN   
management             Mean          1.507625 -0.051061  981.0  88819.0   
                       SD            2.615352       NaN    NaN      NaN   
finance                Mean          1.799865  0.103662  981.0  88819.0   
                       SD            2.612439       NaN    NaN      NaN   
business               Mean          1.367242 -0.362541  981.0  88819.0   
                       SD            2.650816       NaN    NaN      NaN   
data                   Mean          0.325713 -0.548393  981.0  88819.0   
                       SD            2.898043       NaN    NaN      NaN   
software               Mean          0.661524 -0.109194  981.0  88819.0   
                       SD            1.282903       NaN    NaN      NaN   
office                 Mean          0.446545 -0.423297  981.0  88819.0   
                       SD            1.219434       NaN    NaN      NaN   
ml                     Mean          0.685703  0.214220  981.0  88819.0   
                       SD            4.614887       NaN    NaN      NaN   
technical              Mean          0.417419 -0.329342  981.0  88819.0   
                       SD            1.817332       NaN    NaN      NaN   
Reasoning              Mean          1.720976  0.704887  981.0  88819.0   
                       SD            1.752583       NaN    NaN      NaN   
cognitive              Mean          1.678605  0.704638  981.0  88819.0   
                       SD            1.913393       NaN    NaN      NaN   
creativity             Mean          1.947435  0.706213  981.0  88819.0   
                       SD            4.944711       NaN    NaN      NaN   
People                 Mean          1.074125 -0.201700  981.0  88819.0   
                       SD            0.829707       NaN    NaN      NaN   
social                 Mean          1.040283 -0.162043  981.0  88819.0   
                       SD            0.971317       NaN    NaN      NaN   
character              Mean          0.915840 -0.379915  981.0 

## Final table

In [74]:
table = pandas.merge(left=table1,right=table2,how='inner',left_on=['annotation','statistics'],right_on=['annotation','statistics'])
table[('Manager','Within')] = (table[('Manager','N-TM')]*table[('Manager','AM-TM')])/table[('All','N-TM')]
table[('Manager','Between')] = (table[('Manager','N-AM')]/table[('All','N-AM')]-table[('Manager','N-TM')]/table[('All','N-TM')])*table[('Manager','AM')]
table[('Engineer','Within')] = (table[('Engineer','N-TM')]*table['Engineer','AM-TM'])/table['All','N-TM']
table[('Engineer','Between')] = (table[('Engineer','N-AM')]/table[('All','N-AM')]-table[('Engineer','N-TM')]/table[('All','N-TM')])*table[('Engineer','AM')]
table[('Technician','Within')] = (table[('Technician','N-TM')]*table['Technician','AM-TM'])/table['All','N-TM']
table[('Technician','Between')] = (table[('Technician','N-AM')]/table[('All','N-AM')]-table[('Technician','N-TM')]/table[('All','N-TM')])*table[('Technician','AM')]
table[('Operator','Within')] = (table[('Operator','N-TM')]*table['Operator','AM-TM'])/table['All','N-TM']
table[('Operator','Between')] = (table[('Operator','N-AM')]/table[('All','N-AM')]-table[('Operator','N-TM')]/table[('All','N-TM')])*table[('Operator','AM')]
table[('All','Within-Check')] = table[('Manager','Within')]+table[('Engineer','Within')]+table[('Technician','Within')]+table[('Operator','Within')]
table[('All','Between')] = table[('Manager','Between')]+table[('Engineer','Between')]+table[('Technician','Between')]+table[('Operator','Between')]
table = numpy.trunc(1000*table)/1000
table[('All','Within')] = table[('All','AM')] - table[('All','TM')] - table['All','Between']
check = table[[('All','Within-Check'),('All','Within')]]
print(check) # because of decimal places; 'Within-Check' is calculated from the occupations, 'Within' is calculated from All to deal with decimal places
mylist = [('All','AM'),('All','TM'),('All','Between'),('All','Within'),('Manager','AM'),('Manager','TM'),('Engineer','AM'),('Engineer','TM'),('Technician','AM'),('Technician','TM'),('Operator','AM'),('Operator','TM')]
table[mylist] = table[mylist].applymap('{:.3f}'.format)
for i in ['All','Manager','Engineer','Technician','Operator']:
  table.loc[(table[(i,'p-val')]<0.1)&(table[(i,'p-val')]>=0.05),(i,'AM')] = table.loc[(table[(i,'p-val')]<0.1)&(table[(i,'p-val')]>=0.05),(i,'AM')]+'*'
  table.loc[(table[(i,'p-val')]<0.05)&(table[(i,'p-val')]>=0.01),(i,'AM')] = table.loc[(table[(i,'p-val')]<0.05)&(table[(i,'p-val')]>=0.01),(i,'AM')]+'**'
  table.loc[table[(i,'p-val')]<0.01,(i,'AM')] = table.loc[table[(i,'p-val')]<0.01,(i,'AM')]+'***'
table = table[mylist]
table.loc[pandas.IndexSlice[:,'SD'],:] = table.loc[pandas.IndexSlice[:,'SD'],:].apply(lambda x: '(' + x + ')')
table = table.mask(table == '(nan)',numpy.nan)
table = table.mask(table == 'nan',numpy.nan)
table = table.mask(table == '-0.000','0.000')
table.loc[pandas.IndexSlice['Number of job postings',:],:] = table.loc[pandas.IndexSlice['Number of job postings',:],:].replace(r'.000','',regex=True)
table.loc[pandas.IndexSlice['Number of plants',:],:] = table.loc[pandas.IndexSlice['Number of plants',:],:].replace(r'.000','',regex=True)
table

                                           All       
                                  Within-Check Within
annotation             statistics                    
Engineering            Mean              0.444  0.445
                       SD                  NaN    NaN
development            Mean              0.377  0.378
                       SD                  NaN    NaN
design                 Mean              0.330  0.331
                       SD                  NaN    NaN
materials              Mean              0.795  0.797
                       SD                  NaN    NaN
Operations             Mean             -0.026 -0.027
                       SD                  NaN    NaN
inventory              Mean             -0.389 -0.390
                       SD                  NaN    NaN
tooling                Mean             -0.351 -0.351
                       SD                  NaN    NaN
automation             Mean              1.588  1.589
                       SD   

All                            \
                                         AM        TM Between  Within   
annotation             statistics                                       
Engineering            Mean        1.796***     1.011   0.340   0.445   
                       SD           (1.506)   (1.314)     NaN     NaN   
development            Mean        1.653***     1.005   0.270   0.378   
                       SD           (2.125)   (1.796)     NaN     NaN   
design                 Mean        1.651***     0.982   0.338   0.331   
                       SD           (2.083)   (1.884)     NaN     NaN   
materials              Mean        2.345***     1.080   0.468   0.797   
                       SD           (4.219)   (3.016)     NaN     NaN   
Operations             Mean        1.138***     1.051   0.114  -0.027   
                       SD           (1.122)   (1.329)     NaN     NaN   
inventory              Mean        0.568***     0.983  -0.025  -0.390   
                       SD           (1.455)   (2.293)     NaN     NaN   
tooling                Mean           1.445     1.315   0.481  -0.351   
                       SD          (26.030)  (19.407)     NaN     NaN   
automation             Mean        2.898***     1.022   0.287   1.589   
                       SD           (6.862)   (4.822)     NaN     NaN   
production             Mean        1.559***     1.030   0.189   0.340   
                       SD           (1.927)   (1.970)     NaN     NaN   
maintenance            Mean        0.753***     1.168   0.114  -0.529   
                       SD           (1.679)   (2.850)     NaN     NaN   
Support                Mean        0.771***     0.950  -0.100  -0.079   
                       SD           (0.668)   (0.783)     NaN     NaN   
administrative         Mean        1.371***     1.103  -0.550   0.818   
                       SD           (3.712)   (3.143)     NaN     NaN   
management             Mean        0.863***     1.020  -0.201   0.044   
                       SD           (1.955)   (2.093)     NaN     NaN   
finance                Mean        0.753***     0.971  -0.331   0.113   
                       SD           (1.699)   (2.229)     NaN     NaN   
business               Mean        0.504***     0.989  -0.234  -0.251   
                       SD           (1.588)   (2.355)     NaN     NaN   
data                   Mean        0.594***     0.979   0.080  -0.465   
                       SD           (2.916)   (4.488)     NaN     NaN   
software               Mean        0.708***     0.867   0.039  -0.198   
                       SD           (1.224)   (1.687)     NaN     NaN   
office                 Mean        0.762***     0.932   0.059  -0.229   
                       SD           (1.526)   (1.858)     NaN     NaN   
ml                     Mean        1.196***     0.782   0.173   0.241   
                       SD           (5.493)   (5.070)     NaN     NaN   
technical              Mean        0.615***     0.997   0.042  -0.424   
                       SD           (2.302)   (3.528)     NaN     NaN   
Reasoning              Mean        1.390***     0.980  -0.103   0.513   
                       SD           (1.489)   (1.459)     NaN     NaN   
cognitive              Mean        1.371***     0.985  -0.101   0.487   
                       SD           (1.585)   (1.530)     NaN     NaN   
creativity             Mean        1.492***     0.953  -0.111   0.650   
                       SD           (4.150)   (3.439)     NaN     NaN   
People                 Mean        0.852***     0.989  -0.072  -0.065   
                       SD           (0.688)   (0.776)     NaN     NaN   
social                 Mean        0.929***     1.007  -0.029  -0.049   
                       SD           (0.954)   (1.056)     NaN     NaN   
character              Mean        0.749***     0.970  -0.054  -0.167   
                       SD           (1.040)   (1.227)     NaN     NaN   
customer               Me

In [75]:
table.to_excel('Appendix-Table3-terms.xlsx')